In [ ]:
import numpy as np
from coverage import *

import geopandas as gpd
import pandas as pd
import branca
import folium


In [ ]:
num_days = 8
tles = gen_sats(sat_nos=[39084,49260])
times = gen_times(start_yr=2021, start_mo=12, start_day=10, days=num_days, step_min=1)

# Generate the instrument from basic specs
# inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=1800, mm=True) # Actual TIRS
inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=4000, mm=True) # Adjusted TIRS for 1 min FOVs

# Select cell size for coverage map
xcell_size = ycell_size = 0.5

## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# aoi =  world[world.name == "Brazil"].geometry
aoi =  world[world.continent == "South America"].geometry
# aoi = world[world.name == "United States of America"]#.geometry

In [ ]:
# Batch FOV generation over N satellites

gdfs = []
for tle in tles:
    sat = tle[0]
    poly_df = forecast_fovs(sat, times, inst)
    gdfs.append(poly_df)

poly_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="epsg:4326")
poly_df["lonspan"] = poly_df.bounds['maxx'] - poly_df.bounds['minx']

# Create cmap for unique satellites and create color column
sat_ids = list(poly_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)
poly_df['color'] = poly_df['id'].apply(cmap)

In [ ]:
# Filter shapes crossing anti-meridian
plot_df = poly_df[poly_df["lonspan"] < 20].copy()

# Filter by AOI
xmin, ymin, xmax, ymax= aoi.total_bounds
plot_df = plot_df.cx[xmin: xmax, ymin: ymax]

m = plot_df.explore(color="color", tooltip=["satellite", "time"])
m#.save("fov_output.html")

In [ ]:
## Coverage data analysis for single satellite/ batch of satellites
# 1) Create a grid of equally spaced points

grid, grid_shape = create_grid(aoi.total_bounds, xcell_size, ycell_size)


# 2) Add "n_visits" column to grid using sjoin/ dissolve
shapes = gpd.GeoDataFrame(plot_df.geometry)
merged = gpd.sjoin(shapes, grid, how='left', predicate="intersects")
merged['n_visits']=0 # this will be replaced with nan or positive int where n_visits > 0
dissolve = merged.dissolve(by="index_right", aggfunc="count") # no difference in count vs. sum here?
grid.loc[dissolve.index, 'n_visits'] = dissolve.n_visits.values

grid.n_visits.describe()


In [ ]:
# Plotting
# Form 2D array of n_visits based on grid shape
img = np.rot90(grid.n_visits.values.reshape(grid_shape))

# Create colormap and apply to img
colormap = branca.colormap.step.viridis.scale(1, grid.n_visits.max())
# colormap = branca.colormap.step.viridis.scale(1, 2)

def colorfunc(x):
    if np.isnan(x):
        return (0,0,0,0)
    else:
        return colormap.rgba_bytes_tuple(x)

# Apply cmap to img array and rearrange for RGBA
cmap = np.vectorize(colorfunc)
rgba_img = np.array(cmap(img))
rgba_img = np.moveaxis(rgba_img, 0, 2)

# Update image corner bounds based on cell size
xmin, ymin, xmax, ymax= grid.total_bounds
xmin = xmin - xcell_size/2
ymin = ymin - ycell_size/2
xmax = xmax + xcell_size/2
ymax = ymax + ycell_size/2

m = folium.Map()
m.fit_bounds([[ymin, xmin], [ymax, xmax]])
m.add_child(folium.raster_layers.ImageOverlay(rgba_img, opacity=.4, mercator_project=True,# crs="EPSG:4326",
                                 bounds = [[ymin,xmin],[ymax,xmax]]))
colormap.add_to(m)
m#.save("coverage_output.html")
